### Notebook to adjust PetroNER for Entity Linking task

In [27]:
from owlready2 import *
import gensim
from conllu import parse_incr, parse
import numpy as np

Load ontology and embeddings

In [28]:
#OntoGeoLógica populada (OntoGeoLogicaPopulada.owl)
onto = get_ontology("../../KnowledgeGraph/OntoGeoLogicaPovoadaInstanciasRelacoes.owl")
onto.load()

get_ontology("http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#")

In [29]:
# Modelo OWL2Vec - 
PetroOntoVec = gensim.models.Word2Vec.load("../../Embeddings/PetroOntoVec/PetroOntoVec_simples/outputontology.embeddings")

Load PetroNER (file in CONLLU format)

In [30]:
# CONLLU parser
boleltins = open("../PetroNER/petroner-uri-2022-03-07.conllu", "r", encoding="utf-8")
sentences = parse_incr(boleltins)

### Preparing and processing the dataset

Função que recebe o texto anotado em formato CONLLU e processa para o treinamento. Tem como saída as sentenças acrescida das tags para indicar as entidades, bem como com a entidade no início do texto; lista com as entidades (instâncias extraídas do texto); lista com as classes que as entidades pertencem; e lista com as URI referente às entidades. O texto sai com o seguinte formato:  
"[Nome da classe] Nome da entidade | blablabla blablabla blablabla [E] Nome da entidade [/E] blablabla blablabla." 

In [31]:
# Função que recebe
def preparar_dataset(tokenlist):

    #sent = TokenList([])
    sent_orig = ''   # String para compor a sentença original
    sents_NER = []   # Lista para armazenar as sentenças com indicação das entidades
    NER_on = False   # Variável para indicar se está iterando por token de entidades
    NER_tok = ''     # string para armazenar os tokens das entidades que estão sendo iteradas
    MW_on = False    # Variável para indicar se está iterando por tolkens multiword
    MW_tok = ''      # string para armazenar o termo multiword
    MW_end = 0       # ID final do token multiword
    ENT = []         # Lista para armazenar as entidades encontradas no texto
    ENT_class = []   # Lista para armazenar as classes das entidades encontradas no texto
    ENT_URI = []     # Lista para armazenar as URI das entidades encontradas no texto

    #Iterando por todos os tokens de uma sentença
    for tok in tokenlist:
        # ignorando id não inteiro (tuplas), ou seja token multiword
        if type(tok['id']) == int:

            # Verifica se o token inicia uma entidade ("B")
            if tok['deps'][0] == 'B':
                
                # Verifica se a variável NER_on está ativado (No caso de um tok "B" seguido de outro)
                if NER_on == True:
                    
                    NER_on = False  # Desativa a variável NER_on

                    # Para sentença mais recente da lista sents_NER, inclui a entidade no início e o marcador [/E] no final 
                    sents_NER[-1] = '[' + ENT_class[-1] + '] ' + NER_tok + '| ' + sents_NER[-1].strip() + ' [/E]'

                    # Incluir o NER_tok na lista ENT
                    ENT.append(NER_tok.strip())

                    NER_tok = '' # Apaga a string NER_tok

                NER_on = True  # Ativa a variável NER_on
                # Acrescenta uma sentença com o marcador [E] na listas de sentenças com NER
                sents_NER.append(sent_orig + ' [E]')
                # Incluir classe na lista ENT_class
                ENT_class.append(tok['deps'][2:])
                # Incluir URI da entidade na lista ENT_URI. Caso não haja URI, incluir "None". 
                try:
                    ENT_URI.append(tok['misc']['grafo'])
                except:
                    ENT_URI.append(None)

            # Identifica o primeiro token "O" após uma sequência de entidades
            if (tok['deps'][0] == 'O') & NER_on:
                
                NER_on = False  # Desativa a variável NER_on

                # Para sentença mais recente da lista sents_NER, 
                # inclui a classe e a entidade no início e o marcador [/E] no final 
                sents_NER[-1] = '[' + ENT_class[-1] + '] ' + NER_tok + '| ' + sents_NER[-1].strip() + ' [/E]'

                # Incluir o NER_tok na lista ENT
                ENT.append(NER_tok.strip())

                NER_tok = '' # Apaga a string NER_tok
                
            # Identifica se o último token da sequência tem o marcador "B" ou "I".    
            if (tokenlist[-1]['id'] == tok['id']) & ((tok['deps'][0] == 'B') | (tok['deps'][0] == 'I')):
                NER_on = False  # Desativa a variável NER_on

                # Para sentença mais recente da lista sents_NER, 
                # inclui a classe e a entidade no início e o marcador [/E] no final 
                sents_NER[-1] = '[' + ENT_class[-1] + '] ' + NER_tok + '| ' + sents_NER[-1].strip() + ' [/E]'

                # Incluir o NER_tok na lista ENT
                ENT.append(NER_tok.strip())

                NER_tok = '' # Apaga a string NER_tok

            # Verifica se está iterando por uma multiword ou não
            if not MW_on:

                # Acrecenta o token à string da entidade
                if NER_on:
                    NER_tok = NER_tok + tok['form'] + ' '

                # Se o token for uma pontuação, não inclui espaço entre o token anterior 
                if tok['upos'] == 'PUNCT':
                    # O token é incluído na senteça original e em todas as sentenças da lista sents_NER
                    sent_orig = sent_orig + tok['form']
                    for i in range(len(sents_NER)):
                        sents_NER[i] = sents_NER[i] + tok['form']
                else:
                    sent_orig = sent_orig + ' ' + tok['form']
                    for i in range(len(sents_NER)):
                        sents_NER[i] = sents_NER[i] + ' ' + tok['form']

            else:
                # Se o token for o último termo da multiword, desativamos a variável MW_on
                if tok['id'] == MW_end:
                    MW_on = False

                    # Acrecenta a multiword à string da entidade
                    if NER_on:
                        NER_tok = NER_tok + MW_tok + ' '

                    #Ao invés de incluir nas senteças o token, vamos incluir o MW_tok 
                    # Se o token for uma pontuação, não inclui espaço entre o token anterior 
                    if tok['upos'] == 'PUNCT':
                        # O token é incluído na senteça original e em todas as sentenças da lista sents_NER
                        sent_orig = sent_orig + MW_tok
                        for i in range(len(sents_NER)):
                            sents_NER[i] = sents_NER[i] + MW_tok
                    else:
                        sent_orig = sent_orig + ' ' + MW_tok
                        for i in range(len(sents_NER)):
                            sents_NER[i] = sents_NER[i] + ' ' + MW_tok

        # Se a ID do token for referente a uma multiword, ativamos a variável MW_on, 
        # armazenamos o termo MW_tok e o valor da 'id' final
        else:
            MW_on = True
            MW_tok = tok['form']
            MW_end = tok['id'][-1]
    
    return(sents_NER, ENT, ENT_class, ENT_URI)

In [32]:
# Separando as listas de sentenças, entidades, classes e URI
text = []
entities = []
classes_NER = [] 
ENT_URI = []

# Iterando por todas as sentenças em formato CONLLU para preparar o dataset
n = 0
for tokenlist in sentences:
    n = n + 1

    text_n, entities_n, classes_NER_n, ENT_URI_n = preparar_dataset(tokenlist)
    
    text = text + text_n
    entities = entities + entities_n
    classes_NER = classes_NER + classes_NER_n
    ENT_URI = ENT_URI + ENT_URI_n
    
print('Total de sentenças no arquivo: ', n)
print('N° de sentenças com entidades: ', len(text))
print ('N° de entidades: ', len(entities))
print('N° de classes: ', len(classes_NER))
print('N° de classes: ', len(ENT_URI))

Total de sentenças no arquivo:  24035
N° de sentenças com entidades:  18749
N° de entidades:  18749
N° de classes:  18749
N° de classes:  18749


In [33]:
count = 0
Ent_sem_URI = []
for i in range(len(ENT_URI)):
    if ENT_URI[i] == None:
        count = count + 1
        Ent_sem_URI.append(entities[i])

print('Número de entidades sem URI: ', count)
print('Entidades sem URI: ')
print(Ent_sem_URI)      

Número de entidades sem URI:  5647
Entidades sem URI: 
['Santana', 'Membro Mucuri', 'hidrocarbonetos', 'selo', 'petróleo', 'água', 'poços', 'petróleo', 'poços', 'poços', 'Bacia do', 'Bacia do Tucano', 'água', 'água do mar', 'Bacia do Tucano', 'poços', 'selo', 'petróleo', 'Bonsucesso', 'poços', 'poços', 'Alagoas', 'Potiguar', 'Ceará', 'Potiguar', 'Campos', 'Espirito Santo', 'Campos', 'água do mar', 'rocha', 'águas', 'água do mar', 'rocha', 'águas', 'Bacia', 'Campos', 'água', 'Bacia', 'aptiano', 'cenomaniano', 'mesocenomaniano', 'Alagoas', 'Potiguar', 'água', 'água', 'mesocenomaniano', 'Alagoas', 'poço', 'Potiguar', 'Ceará', 'Alagoas', 'Potiguar', 'Potiguar', 'Ceará', 'poços', 'Espirito Santo', 'Campos', 'Alagoas', 'Potiguar', 'Campos', 'Espirito Santo', 'Campos', 'Campos', 'poços', 'poços', 'poços', 'cenomaniano', 'santoniano', 'cenomaniano', 'santoniano', 'rocha', 'turoniano', 'santoniano', 'coniaciano', 'santoniano', 'neossantoniano', 'eocampaniano', 'Neoconiaciano', 'eomaastrichtiano

### Buscando vetores do PetroOntoVec para cada URI

In [34]:
# Separando as listas de sentenças, entidades, classes, URI e vetores
text_new = []
entities_new = []
classes_NER_new = [] 
ENT_URI_new = []
URI_Vec_new = []

URI_ausente_Onto = []

n = 0 
#m = 0
#Iterando por todas as sentenças, desconsiderando aquelas cujas entidades não possuem URI
for i in range(len(text)):
    n = n + 1
    if ENT_URI[i] != None:
        # Quebrando as entidades que possuem mais de um URI (ex: "#Aptian,#Miocene" e "#Cenozoic,#Mesozoic")
        ENT_URI_n = ENT_URI[i].split(',#')
        
        # Iterando pelas ENT_URI após serem quebradas e acrescentando '#" naquelas que perderam na operação de split 
        for ent in ENT_URI_n:
            if ent[0] != '#':
                ent = '#' + ent
            try:
                #Acrescentar a URI da entidade e o vetor da URI nas listas
                URI_Vec_new.append(PetroOntoVec['http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2' + ent])
                ENT_URI_new.append(ent)
               
                # O texto, a entidade no texto e a classe são repetidas quando existem multiplas URI para uma mesma entidade
                text_new.append(text[i])
                entities_new.append(entities[i])
                classes_NER_new.append(classes_NER[i])
            except:
                #print("URI presente no arquivo CONLLU mas ausente na Ontologia: ", ent)
                URI_ausente_Onto.append(ent)

# Transformando as listas em Numpy array
text = np.array(text_new)
entities = np.array(entities_new)
classes_NER = np.array(classes_NER_new) 
ENT_URI = np.array(ENT_URI_new)
URI_Vec = np.array(URI_Vec_new)
            
print('Total de sentenças anotadas: ', n)
print('N° de sentenças: ', len(text))
print ('N° de entidades: ', len(entities))
print('N° de classes: ', len(classes_NER))
print('N° de entidades_ID: ', len(ENT_URI))       
print('N° de vetores de URI: ', len(URI_Vec))       

Total de sentenças anotadas:  18749
N° de sentenças:  13045
N° de entidades:  13045
N° de classes:  13045
N° de entidades_ID:  13045
N° de vetores de URI:  13045
URI ausentes da ontologia: 
{'#inter_membro_002', '#inter_formacao_007', '#INTER_BACIA_023', '#INTER_BACIA_004', '#Rio_da_serra_Age', '#inter_membro_003', '#INTER_BACIA_002', '#INTER_BACIA_005', '#INTER_BACIA_017', '#inter_membro_008', '#INTER_BACIA_008', '#INTER_BACIA_022', '#inter_membro_004', '#grupo_05', '#INTER_BACIA_020', '#inter_membro_011', '#INTER_BACIA_013', '#inter_formacao_015', '#INTER_BACIA_019', '#Eoriodaserra_Age', '#INTER_BACIA_007', '#INTER_CAMPO_0001', '#inter_membro_010', '#inter_formacao_004', '#Kasimovian / #Gzhelian', '#inter_membro_005', '#INTER_BACIA_021', '#INTER_BACIA_024', '#inter_membro_012', '#INTER_BACIA_018', '#INTER_BACIA_011', '#inter_membro_007', '#inter_formacao_016', '#INTER_BACIA_025', '#INTER_BACIA_006', '#inter_formacao_005', '#INTER_BACIA_010', '#INTER_BACIA_003', '#INTER_CAMPO_0002', '

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:25: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [35]:
print('URI ausentes da ontologia: ')
set(URI_ausente_Onto)

URI ausentes da ontologia: 


{'#Eoriodaserra_Age',
 '#INTER_BACIA_001',
 '#INTER_BACIA_002',
 '#INTER_BACIA_003',
 '#INTER_BACIA_004',
 '#INTER_BACIA_005',
 '#INTER_BACIA_006',
 '#INTER_BACIA_007',
 '#INTER_BACIA_008',
 '#INTER_BACIA_009',
 '#INTER_BACIA_010',
 '#INTER_BACIA_011',
 '#INTER_BACIA_012',
 '#INTER_BACIA_013',
 '#INTER_BACIA_014',
 '#INTER_BACIA_015',
 '#INTER_BACIA_016',
 '#INTER_BACIA_017',
 '#INTER_BACIA_018',
 '#INTER_BACIA_019',
 '#INTER_BACIA_020',
 '#INTER_BACIA_021',
 '#INTER_BACIA_022',
 '#INTER_BACIA_023',
 '#INTER_BACIA_024',
 '#INTER_BACIA_025',
 '#INTER_BACIA_026',
 '#INTER_CAMPO_0001',
 '#INTER_CAMPO_0002',
 '#INTER_CAMPO_003',
 '#Kasimovian / #Gzhelian',
 '#Mesoriodaserra_Age',
 '#Neo-Alagoas_Subage',
 '#Permiano',
 '#Rio_da_serra_Age',
 '#carbonatos',
 '#dry_hole',
 '#gipsita',
 '#grupo_05',
 '#inter_formacao_001',
 '#inter_formacao_002',
 '#inter_formacao_003',
 '#inter_formacao_004',
 '#inter_formacao_005',
 '#inter_formacao_006',
 '#inter_formacao_007',
 '#inter_formacao_008',
 '#int

### Salvando dataset

In [9]:
with open('sentences.npy', 'wb') as f:
    np.save(f, text)
with open('entities.npy', 'wb') as f:
    np.save(f, entities)
with open('classes.npy', 'wb') as f:
    np.save(f, classes_NER)
with open('URI.npy', 'wb') as f:
    np.save(f, ENT_URI)
with open('URI_vectors.npy', 'wb') as f:
    np.save(f, URI_Vec)